In [1]:
import numpy as np
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")

PyTorch 版本: 2.9.1
CUDA 可用: False


In [ ]:
#获取data
train_data = pd.read_csv('./covid.train_new.csv').values  # values 是属性，不是方法，不需要括号
x_train = train_data[:,:-1]
y_train = train_data[:,-1]

class Covid19Dataset(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

covid_dataset = Covid19Dataset(x_train, y_train)
train_loader = DataLoader(covid_dataset, batch_size=32, shuffle=True, pin_memory=True)

class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze()
        return x    


model = MyModel(input_dim=x_train.shape[1]).to('cpu')
criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to('cpu')
        y = y.to('cpu')
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()


torch.save(model.state_dict(), 'model.pth')
ckpt = torch.load('model.pth')
model.load_state_dict(ckpt)


        

/Users/jianbopan/miniforge3/envs/llm/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/jianbopan/miniforge3/envs/llm/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
